<a href="https://colab.research.google.com/github/pakyeon/deep-rl-class-ko/blob/main/notebooks/ko/unit6/unit6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 유닛 6: Panda-Gym 로보틱스 시뮬레이션을 활용한 어드밴티지 액터 크리틱 (A2C) 🤖

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit8/thumbnail.png"  alt="Thumbnail"/>

이 노트북에서는 [Panda-Gym](https://github.com/qgallouedec/panda-gym)을 사용하여 A2C를 사용하는 방법을 배웁니다. 다음과 같은 작업을 수행하도록 **로봇 팔**(Franka Emika Panda 로봇)을 훈련시킬 것입니다:

- `도달(Reach)`: 로봇은 목표 위치에 엔드 이펙터(end-effector)를 놓아야 합니다.

그 후에는 **다른 로보틱스 작업들을 훈련**시킬 수 있게 될 것입니다.

### 🎮 환경:

- [Panda-Gym](https://github.com/qgallouedec/panda-gym)

### 📚 RL 라이브러리:

- [Stable-Baselines3](https://stable-baselines3.readthedocs.io/)

튜토리얼을 지속적으로 개선하고 있으니, **이 노트북에서 문제가 발견되면** [GitHub 저장소에 이슈를 등록해주세요](https://github.com/huggingface/deep-rl-class/issues).

## 이 노트북의 목표 🏆

이 노트북을 마치면 다음을 할 수 있게 됩니다:

* 환경 라이브러리인 **Panda-Gym**을 사용할 수 있습니다.
* **A2C를 사용하여 로봇을 훈련**할 수 있습니다.
* **입력을 정규화해야 하는 이유**를 이해할 수 있습니다.
* 멋진 비디오 재생 및 평가 점수와 함께 **훈련된 에이전트와 코드를 Hub에 푸시**할 수 있습니다 🔥.

## 이 노트북은 심층 강화 학습 과정에서 가져왔습니다.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration"/>

이 무료 과정에서 여러분은 다음을 배우게 됩니다:

  - 📖 심층 강화 학습을 **이론과 실습**으로 학습합니다.
  - 🧑‍💻 Stable Baselines3, RL Baselines3 Zoo, CleanRL 및 Sample Factory 2.0과 같은 **유명한 심층 RL 라이브러리 사용법**을 배웁니다.
  - 🤖 **고유한 환경에서 에이전트**를 훈련시킵니다.

더 많은 내용은 📚 강의 계획서를 확인하세요 👉 https://simoninithomas.github.io/deep-rl-course

**<a href="http://eepurl.com/ic5ZUD">과정에 등록</a>**하는 것을 잊지 마세요 (각 유닛이 게시될 때 링크를 보내드리고 챌린지 및 업데이트에 대한 정보를 제공하기 위해 이메일을 수집합니다).

가장 좋은 소통 방법은 디스코드 서버에 가입하여 커뮤니티 및 우리와 교류하는 것입니다 👉🏻 https://discord.gg/ydHrjt3WP5

## 준비 사항 🏗️
노트북을 시작하기 전에 다음을 수행해야 합니다.

🔲 📚 [유닛 6을 읽고 액터-크리틱 방법론](https://huggingface.co/deep-rl-course/unit6/introduction)을 학습하세요 🤗

# 첫 번째 로봇을 훈련시켜 봅시다 🤖

이 실습을 [인증 절차](https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process)에 대해 검증하려면, 훈련된 모델을 Hub에 푸시하고 다음 결과를 얻어야 합니다.

- `PandaReachDense-v3` 결과가 >= -3.5여야 합니다.

결과를 확인하려면 [리더보드](https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard)로 이동하여 모델을 찾으세요. **결과 = 평균 보상 - 보상 표준 편차**

인증 절차에 대한 자세한 내용은 이 섹션을 확인하세요 👉 https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process

## GPU 설정하기 💪  
- **에이전트의 훈련을 가속하기 위해 GPU를 사용할 예정입니다**. 이를 위해 `런타임 > 런타임 유형 변경`으로 이동하세요.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step1.jpg" alt="GPU Step 1">

- `하드웨어 가속기 > GPU`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step2.jpg" alt="GPU Step 2">

## 가상 디스플레이 생성 🔽

노트북을 실행하는 동안 리플레이 비디오를 생성해야 합니다. 이를 위해 코랩에서는 **환경을 렌더링(그리고 프레임을 기록)할 수 있도록 가상 화면이 필요합니다.**

따라서 다음 셀은 라이브러리를 설치하고 가상 화면 🖥을 생성 및 실행합니다.

In [ ]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

### 종속성 설치 🔽

첫 번째 단계는 종속성을 설치하는 것입니다. 여러 개를 설치할 것입니다:
- `gymnasium`
- `panda-gym`: 로봇 팔 환경을 포함합니다.
- `stable-baselines3`: SB3 심층 강화 학습 라이브러리입니다.
- `huggingface_sb3`: Stable-baselines3에서 Hugging Face 🤗 Hub의 모델을 로드하고 업로드하기 위한 추가 코드입니다.
- `huggingface_hub`: 누구나 Hub 리포지토리로 작업할 수 있도록 하는 라이브러리입니다.

⏲ 설치는 **10분** 정도 소요될 수 있습니다.

In [ ]:
!pip install stable-baselines3[extra]
!pip install gymnasium

In [ ]:
!pip install huggingface_sb3
!pip install huggingface_hub
!pip install panda_gym

## 패키지 임포트 📦

In [ ]:
import os

import gymnasium as gym
import panda_gym

from huggingface_sb3 import load_from_hub, package_to_hub

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

from huggingface_hub import notebook_login

## PandaReachDense-v3 🦾

우리가 훈련할 에이전트는 제어(팔 움직임 및 말단 장치 사용)를 수행해야 하는 로봇 팔입니다.

로봇 공학에서 *말단 장치(end-effector)*는 환경과 상호 작용하도록 설계된 로봇 팔 끝에 있는 장치입니다.

`PandaReach`에서 로봇은 목표 위치(녹색 공)에 말단 장치를 놓아야 합니다.

우리는 이 환경의 밀집(dense) 버전을 사용할 것입니다. 이는 **각 타임스텝마다 보상을 제공하는** *밀집 보상 함수(dense reward function)*를 얻게 된다는 것을 의미합니다 (에이전트가 작업을 완료하는 데 가까울수록 보상이 높습니다). 이는 작업이 완료된 경우에만 환경이 보상을 반환하는 *희소 보상 함수(sparse reward function)*와 대조됩니다.

또한, 우리는 *말단 장치 변위 제어(End-effector displacement control)*를 사용할 것입니다. 이는 **액션이 말단 장치의 변위에 해당한다**는 것을 의미합니다. 각 관절의 개별 움직임을 제어하지 않습니다 (관절 제어).

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit8/robotics.jpg"  alt="Robotics"/>

이 방법을 사용하면 **훈련이 더 쉬워질 것입니다**.

### 환경 생성

#### 환경 🎮

`PandaReachDense-v3`에서 로봇 팔은 목표 위치(녹색 공)에 말단 장치를 놓아야 합니다.

In [ ]:
env_id = "PandaReachDense-v3"

# 환경 생성  
env = gym.make(env_id)

# 상태 공간과 행동 공간 가져오기  
s_size = env.observation_space.shape
a_size = env.action_space

In [ ]:
print("_____관측 공간_____ \n")
print("상태 공간의 크기: ", s_size)
print("샘플 관측값", env.observation_space.sample()) # 랜덤 상태 가져오기

관측 공간은 **3개의 다른 요소로 구성된 딕셔너리**입니다:
- `achieved_goal`: (x,y,z) 말단 장치의 현재 위치.
- `desired_goal`: (x,y,z) 말단 장치의 목표 위치.
- `observation`: 말단 장치의 위치 (x,y,z) 및 속도 (vx, vy, vz).

관측값이 딕셔너리이기 때문에, **우리는 MlpPolicy 대신 MultiInputPolicy 정책을 사용해야 합니다.**

In [ ]:
print("\n _____행동 공간_____ \n")
print("행동 공간의 크기: ", a_size)
print("임의의 행동 샘플", env.action_space.sample()) # 랜덤 행동 선택

행동 공간은 3개의 값을 가진 벡터입니다:
- x, y, z 이동 제어

### 관측값 및 보상 정규화

강화 학습에서 좋은 관행 중 하나는 [입력 피처를 정규화](https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html)하는 것입니다.

이를 위해 입력 피처의 이동 평균과 표준 편차를 계산하는 래퍼가 있습니다.

또한 `norm_reward = True`를 추가하여 이 동일한 래퍼로 보상도 정규화합니다.

[이 셀을 채우려면 문서를 확인해야 합니다](https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#vecnormalize)

In [ ]:
env = make_vec_env(env_id, n_envs=4)

# 관측값과 보상을 정규화하기 위해 이 래퍼 추가
env = # TODO: 래퍼를 추가하세요

#### 해결책

In [ ]:
env = make_vec_env(env_id, n_envs=4)

env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

### A2C 모델 생성 🤖

StableBaselines3를 사용한 A2C 구현에 대한 자세한 정보는 다음을 확인하세요: https://stable-baselines3.readthedocs.io/en/master/modules/a2c.html#notes

최적의 매개변수를 찾기 위해 [Stable-Baselines3 팀의 공식 훈련된 에이전트](https://huggingface.co/sb3)를 확인했습니다.

In [ ]:
model = # A2C 모델을 생성하고 최적의 매개변수를 찾아보세요

#### 해결책

In [ ]:
model = A2C(policy = "MultiInputPolicy",
            env = env,
            verbose=1)

### A2C 에이전트 훈련 🏃
- 에이전트를 1,000,000 타임스텝 동안 훈련해 봅시다. Colab에서 GPU를 사용하는 것을 잊지 마세요. 약 25-40분 정도 소요됩니다.

In [ ]:
model.learn(1_000_000)

In [ ]:
# 에이전트를 저장할 때 모델과 VecNormalize 통계 저장
model.save("a2c-PandaReachDense-v3")
env.save("vec_normalize.pkl")

### 에이전트 평가 📈
- 이제 에이전트가 훈련되었으므로, **성능을 확인**해야 합니다.
- Stable-Baselines3는 이를 위한 메서드인 `evaluate_policy`를 제공합니다.

In [ ]:
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# 저장된 통계 불러오기
eval_env = DummyVecEnv([lambda: gym.make("PandaReachDense-v3")])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

# render_mode를 재정의해야 함
eval_env.render_mode = "rgb_array"

# 테스트 시점에는 업데이트하지 않음
eval_env.training = False
# 테스트 시점에는 보상 정규화가 필요하지 않음
eval_env.norm_reward = False

# 에이전트 로드
model = A2C.load("a2c-PandaReachDense-v3")

mean_reward, std_reward = evaluate_policy(model, eval_env)

print(f"평균 보상 = {mean_reward:.2f} +/- {std_reward:.2f}")

### 훈련된 모델을 Hub에 게시 🔥
훈련 후 좋은 결과를 얻은 것을 확인했으니, 이제 한 줄의 코드로 훈련된 모델을 Hub에 게시할 수 있습니다.

📚 라이브러리 문서 👉 https://github.com/huggingface/huggingface_sb3/tree/main#hugging-face--x-stable-baselines3-v20

`package_to_hub`를 사용하면, 이전 유닛들에서 이미 언급했듯이, **에이전트의 성능을 평가하고, 리플레이를 녹화하고, 모델 카드를 생성한 다음 허브에 푸시할 수 있습니다**.

이러한 방식으로:
- **여러분의 작업을 선보일 수 있습니다** 🔥
- **에이전트가 플레이하는 것을 시각적으로 확인할 수 있습니다** 👀
- **다른 사람들이 사용할 수 있는 에이전트를 커뮤니티와 공유할 수 있습니다** 💾
- **리더보드 🏆에 액세스하여 동기들과 비교했을 때 에이전트 성능이 어느 정도인지 확인할 수 있습니다** 👉 https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard

모델을 커뮤니티에 공유하기 위해서는 다음 **3가지 단계를 추가로 진행해야 합니다**:

1️⃣ (아직 안 했다면) Hugging Face 계정 생성 ➡ [https://huggingface.co/join](https://huggingface.co/join)

2️⃣ 로그인한 후, Hugging Face 웹사이트에서 인증 토큰을 저장해야 합니다.  
- 새 토큰 생성: [https://huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)  
- **권한은 write 역할로 설정해야 합니다**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/create-token.jpg" alt="Create HF Token">

- 토큰을 복사합니다  
- 아래 셀을 실행하고 토큰을 붙여넣습니다

In [ ]:
notebook_login()
!git config --global credential.helper store

Google Colab이나 Jupyter Notebook을 사용하지 않는 경우, 아래 명령어를 대신 사용해야 합니다: `huggingface-cli login`

3️⃣ 이제 학습된 에이전트를 🤗 Hub에 업로드할 준비가 되었습니다 🔥`package_to_hub()` 함수를 사용하면 됩니다.

이 환경에서는 **이 셀을 실행하는 데 약 10분**이 소요될 수 있습니다.

In [ ]:
from huggingface_sb3 import package_to_hub

package_to_hub(
    model=model,
    model_name=f"a2c-{env_id}",
    model_architecture="A2C",
    env_id=env_id,
    eval_env=eval_env,
    repo_id=f"ThomasSimonini/a2c-{env_id}", # 사용자 이름을 변경하세요.
    commit_message="Initial commit",
)

## 추가 도전 과제 🏆
가장 좋은 학습 방법은 **직접 시도해보는 것**입니다! `PandaPickAndPlace-v3`를 시도해보는 건 어떨까요?

panda-gym의 더 고급 작업을 시도하고 싶다면 **TQC 또는 SAC**(로봇 공학 작업에 더 적합한 샘플 효율적인 알고리즘)를 사용하여 수행된 작업을 확인해야 합니다. 실제 로봇 공학에서는 단순한 이유로 더 샘플 효율적인 알고리즘을 사용할 것입니다. 시뮬레이션과 달리 **로봇 팔을 너무 많이 움직이면 파손 위험**이 있기 때문입니다.

PandaPickAndPlace-v1 (이 모델은 환경의 v1 버전을 사용합니다): [https://huggingface.co/sb3/tqc-PandaPickAndPlace-v1](https://huggingface.co/sb3/tqc-PandaPickAndPlace-v1)

그리고 여기에서 panda-gym 문서를 확인하는 것을 망설이지 마세요: [https://panda-gym.readthedocs.io/en/latest/usage/train_with_sb3.html](https://panda-gym.readthedocs.io/en/latest/usage/train_with_sb3.html)

다른 에이전트를 훈련하기 위한 단계(선택 사항)를 제공합니다:

1. "PandaPickAndPlace-v3"라는 환경 정의
2. 벡터화된 환경 만들기
3. 관측값 및 보상을 정규화하기 위한 래퍼 추가. [문서 확인](https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#vecnormalize)
4. A2C 모델 생성 (훈련 로그를 출력하기 위해 verbose=1을 잊지 마세요).
5. 100만 타임스텝 동안 훈련
6. 에이전트 저장 시 모델과 VecNormalize 통계 저장
7. 에이전트 평가
8. `package_to_hub`를 사용하여 훈련된 모델을 Hub에 게시 🔥

### 해결책 (선택 사항)

In [ ]:
# 1 - 2
env_id = "PandaPickAndPlace-v3"
env = make_vec_env(env_id, n_envs=4)

# 3
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

# 4
model = A2C(policy = "MultiInputPolicy",
            env = env,
            verbose=1)
# 5
model.learn(1_000_000)

In [ ]:
# 6
model_name = "a2c-PandaPickAndPlace-v3";
model.save(model_name)
env.save("vec_normalize.pkl")

# 7
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# 저장된 통계 불러오기
eval_env = DummyVecEnv([lambda: gym.make("PandaPickAndPlace-v3")])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

# 테스트 시점에는 업데이트하지 않음
eval_env.training = False
# 테스트 시점에는 보상 정규화가 필요하지 않음
eval_env.norm_reward = False

# 에이전트 로드
model = A2C.load(model_name)

mean_reward, std_reward = evaluate_policy(model, eval_env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

# 8
package_to_hub(
    model=model,
    model_name=f"a2c-{env_id}",
    model_architecture="A2C",
    env_id=env_id,
    eval_env=eval_env,
    repo_id=f"ThomasSimonini/a2c-{env_id}", # TODO: 사용자 이름을 변경하세요.
    commit_message="Initial commit",
)

유닛 7에서 만나요! 🔥
## 계속 학습하고, 멋진 상태를 유지하세요 🤗